#1. RNN for baseline model

In [2]:
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence

class IMDBRNN(nn.Module):
  def __init__(self, input_size, embed_dim, hidden_size, padding_idx):
    super().__init__()
    self.embedding = nn.Embedding(input_size, embed_dim, padding_idx=padding_idx)
    self.rnn = nn.RNN(embed_dim, hidden_size=hidden_size, num_layers=1, batch_first=True)
    self.linear = nn.Linear(hidden_size, 1)

  def forward(self, x, lengths):
    x = self.embedding(x) #(B, T, E)
    packed = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False) #shuffle=True in train_set
    out, h = self.rnn(packed) # dimension of h => (1, B, H)
    h_n = h[-1] # h_n -> (B, H)
    logits = self.linear(h_n).squeeze(-1) #(B, 1) -(squeeze(-1))-> (B,)

    return logits

#2. LSTM model

In [3]:
class IMDBLSTM(nn.Module):
  def __init__(self, input_size, embed_dim, hidden_size, padding_idx):
    super().__init__()
    self.embedding = nn.Embedding(input_size, embed_dim, padding_idx=padding_idx)
    self.lstm = nn.LSTM(embed_dim, hidden_size, num_layers=1, batch_first=True)
    self.linear = nn.Linear(hidden_size, 1)

  def forward(self, x, lengths):
    x = self.embedding(x)
    packed = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
    out, (h, c) = self.lstm(packed)
    h_n = h[-1]
    logits = self.linear(h_n).squeeze(-1)

    return logits